<a href="https://colab.research.google.com/github/Karimsingera/CODERHOUSE-PYTHON-/blob/main/Tratamento_das_Bases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Desafio Obrigatório

Tratamento das bases

Aluno: Karim Singer

In [1]:
import pandas as pd
import numpy as np

In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [3]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -10,
	"longitude": -55,
	"hourly": ["temperature_2m", "precipitation_probability", "rain"],
	"daily": ["precipitation_sum", "rain_sum", "precipitation_probability_max"],
	"timezone": "auto",
	"past_days": 92,
	"forecast_days": 16,
	"models": "best_match"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["rain"] = hourly_rain

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy()
daily_rain_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)


Coordinates -10.0°E -54.875°N
Elevation 420.0 m asl
Timezone b'America/Cuiaba' b'-04'
Timezone difference to GMT+0 -14400 s
                    date  temperature_2m  precipitation_probability  rain
0    2023-09-13 04:00:00       24.020500                        0.0   0.0
1    2023-09-13 05:00:00       23.620499                        0.0   0.0
2    2023-09-13 06:00:00       23.370499                        0.0   0.0
3    2023-09-13 07:00:00       23.420502                        0.0   0.0
4    2023-09-13 08:00:00       23.470501                        0.0   0.0
...                  ...             ...                        ...   ...
2587 2023-12-29 23:00:00       24.779999                        NaN   0.0
2588 2023-12-30 00:00:00       23.580000                        NaN   0.0
2589 2023-12-30 01:00:00       23.180000                        NaN   0.0
2590 2023-12-30 02:00:00       23.230000                        NaN   0.0
2591 2023-12-30 03:00:00       23.230000                      

Explorar dados:

In [4]:
daily_dataframe.head()  # Mostrar as primeiras linhas do DataFrame

,date,precipitation_sum,rain_sum,precipitation_probability_max
0,2023-09-13 04:00:00,0.0,0.0,0.0
1,2023-09-14 04:00:00,0.0,0.0,0.0
2,2023-09-15 04:00:00,0.0,0.0,0.0
3,2023-09-16 04:00:00,0.0,0.0,0.0
4,2023-09-17 04:00:00,10.0,0.0,52.0


In [5]:
daily_dataframe.info()  # Obter informações sobre o DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   date                           108 non-null    datetime64[ns]
 1   precipitation_sum              108 non-null    float32       
 2   rain_sum                       108 non-null    float32       
 3   precipitation_probability_max  103 non-null    float32       
dtypes: datetime64[ns](1), float32(3)
memory usage: 2.2 KB


In [6]:
daily_dataframe.dtypes # Obter informações sobre os tipos de dados

date                             datetime64[ns]
precipitation_sum                       float32
rain_sum                                float32
precipitation_probability_max           float32
dtype: object

In [7]:
daily_dataframe.describe()  # Estatísticas descritivas


,precipitation_sum,rain_sum,precipitation_probability_max
count,108.000000,108.000000,103.000000
mean,3.068518,0.793981,38.961166
std,6.320012,5.230401,34.197788
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.550000,0.000000,35.000000
75%,3.950000,0.000000,66.500000
max,47.399998,41.400002,100.000000


# Selecionar e filtrar dados:


In [8]:
coluna = daily_dataframe['precipitation_probability_max']  # Selecionar uma coluna
print (coluna)

0       0.0
1       0.0
2       0.0
3       0.0
4      52.0
       ... 
103     NaN
104     NaN
105     NaN
106     NaN
107     NaN
Name: precipitation_probability_max, Length: 108, dtype: float32


In [9]:
daily_dataframe.duplicated() #Verificação duplicados

0      False
1      False
2      False
3      False
4      False
       ...  
103    False
104    False
105    False
106    False
107    False
Length: 108, dtype: bool

In [10]:
# Renomear colunas:

daily_dataframe.rename(columns={'date': 'data'}, inplace=True)
daily_dataframe.rename(columns={'precipitation_sum': 'sum_precipitação'}, inplace=True)
daily_dataframe.rename(columns={'rain_sum': 'sum_chuva'}, inplace=True)
daily_dataframe.rename(columns={'precipitation_probability_max': 'chance_de_precipitação_max'}, inplace=True)

print(daily_dataframe.head())

                 data  sum_precipitação  sum_chuva  chance_de_precipitação_max
0 2023-09-13 04:00:00               0.0        0.0                         0.0
1 2023-09-14 04:00:00               0.0        0.0                         0.0
2 2023-09-15 04:00:00               0.0        0.0                         0.0
3 2023-09-16 04:00:00               0.0        0.0                         0.0
4 2023-09-17 04:00:00              10.0        0.0                        52.0


# Lidar com valores nulos:

In [11]:
daily_dataframe.isnull()  # Identificar valores nulos

,data,sum_precipitação,sum_chuva,chance_de_precipitação_max
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
103,False,False,False,True
104,False,False,False,True
105,False,False,False,True
106,False,False,False,True


In [12]:
daily_dataframe_semnull = daily_dataframe.dropna()  # Excluir linhas com valores nulos

In [13]:
daily_dataframe_semnull.isnull() # verificação

,data,sum_precipitação,sum_chuva,chance_de_precipitação_max
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
98,False,False,False,False
99,False,False,False,False
100,False,False,False,False
101,False,False,False,False


In [14]:
daily_dataframe.sort_values(by='chance_de_precipitação_max', ascending=False)  # Ordenar em ordem decrescente

,data,sum_precipitação,sum_chuva,chance_de_precipitação_max
54,2023-11-06 04:00:00,6.500000,0.000000,100.0
71,2023-11-23 04:00:00,15.299999,0.000000,100.0
85,2023-12-07 04:00:00,4.900000,0.000000,100.0
66,2023-11-18 04:00:00,8.400000,0.000000,100.0
69,2023-11-21 04:00:00,1.900000,0.000000,97.0
...,...,...,...,...
103,2023-12-25 04:00:00,0.900000,0.450000,NaN
104,2023-12-26 04:00:00,0.300000,0.150000,NaN
105,2023-12-27 04:00:00,3.900000,3.150000,NaN
106,2023-12-28 04:00:00,30.599998,41.400002,NaN
